In [9]:
### importar librerias
# importar librerias tratamiento de datos
import pandas as pd
import numpy as np
import sklearn as sk
import warnings
import math as m
warnings.filterwarnings('ignore')
import itertools
import time
tiempo=time.time()


# librerias pre-procesamiento
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, RobustScaler, PowerTransformer, StandardScaler
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE, KMeansSMOTE
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.utils import shuffle


# librerias para modelos 
from sklearn.svm import SVC
from sklearn.decomposition import KernelPCA, PCA
import sklearn
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV



# metricas de evaluacion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, balanced_accuracy_score
from sklearn.metrics import roc_curve, auc, precision_recall_curve


start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0009944438934326172 seconds ---


# Funciones Auxiliares

In [3]:
# Función para estandarizar las variables continuas
def estandarizacion(df):
    return df.apply(lambda x: (x-x.mean())/x.std(), axis=0)

# Cálculo de IV #Tomada del material del curso Machine Learning Profesor Fernando Lozano.
def calc_iv(df, feature, target, pr=False):
    lst = []
    df[feature] = df[feature].fillna("NULL")

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature,                                                        # Variable
                    val,                                                            # Value
                    df[df[feature] == val].count()[feature],                        # All
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # Good (think: Fraud == 0)
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]]) # Bad (think: Fraud == 1)

    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])
    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])
    
    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])

    data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
    data.index = range(len(data.index))

    if pr:
        print(data)
        print('IV = ', data['IV'].sum())

    iv = data['IV'].sum()
    print('El IV de esta variable es:',iv)
    print(df[feature].value_counts())
    return iv, data

# Matriz de Confusión  #Tomada del material del curso Machine Learning Profesor Fernando Lozano.
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Etiqueta')
    plt.xlabel('Predicción')
    
# Codificación One-Hot #Tomada del material del curso Machine Learning Profesor Fernando Lozano.
def convert_dummy(df, feature,rank=0):
    pos = pd.get_dummies(df[feature], prefix=feature)
    mode = df[feature].value_counts().index[rank]
    biggest = feature + '_' + str(mode)
    pos.drop([biggest],axis=1,inplace=True)
    df.drop([feature],axis=1,inplace=True)
    df=df.join(pos)
    return df

def convert_dummy2(df):
    df = pd.get_dummies(df, drop_first=False)
    return df

#Tomada del material del curso Machine Learning Profesor Fernando Lozano.
def get_category(df, col, binsnum, labels, prefijo, qcut = False):
    if qcut:
        localdf = pd.qcut(df[col], q = binsnum, labels = labels) # quantile cut
    else:
        localdf = pd.cut(df[col], bins = binsnum, labels = labels) # equal-length cut
        
    localdf = pd.DataFrame(localdf)
    name = prefijo + '_' + col
    localdf[name] = localdf[col]
    df = df.join(localdf[name])
    df[name] = df[name].astype(object)
    return df

#  Cargar bases

In [6]:
#Se cargar bases de datos de fraude pre-procesada en R
fraude_preprocess = pd.read_csv("data/fraude_preprocess.csv")
fraude_preprocess

,Unnamed: 0,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,1,-1.996580,-0.694241,-0.044075,1.672771,0.973364,-0.245116,0.347067,0.193679,0.082637,...,-0.024923,0.382854,-0.176911,0.110507,0.246585,-0.392170,0.330891,-0.063781,0.244964,0
1,2,-1.996580,0.608495,0.161176,0.109797,0.316522,0.043483,-0.061820,-0.063700,0.071253,...,-0.307376,-0.880075,0.162201,-0.561130,0.320693,0.261069,-0.022256,0.044607,-0.342474,0
2,3,-1.996558,-0.693499,-0.811576,1.169466,0.268231,-0.364571,1.351451,0.639775,0.207372,...,0.337631,1.063356,1.456317,-1.138090,-0.628536,-0.288446,-0.137137,-0.181021,1.160684,0
3,4,-1.996558,-0.493324,-0.112169,1.182514,-0.609726,-0.007469,0.936148,0.192070,0.316017,...,-0.147443,0.007267,-0.304776,-1.941024,1.241902,-0.460217,0.155396,0.186188,0.140534,0
4,5,-1.996537,-0.591329,0.531540,1.021410,0.284655,-0.295015,0.071998,0.479301,-0.226510,...,-0.012839,1.100009,-0.220123,0.233250,-0.395201,1.041609,0.543619,0.651815,-0.073403,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,284803,1.641929,-6.065831,6.099275,-6.486233,-1.459638,-3.886604,-1.956687,-3.975621,6.116562,...,0.290602,0.154146,1.624571,-0.840999,2.756316,0.518499,2.337897,2.495525,-0.350150,0
284803,284804,1.641950,-0.374121,-0.033356,1.342142,-0.521651,0.629039,0.794444,0.019667,0.246886,...,0.291625,1.273779,0.019958,-1.677917,-1.163724,-0.819645,0.169641,-0.162163,-0.254116,0
284804,284805,1.641971,0.980022,-0.182433,-2.143201,-0.393983,1.905830,2.275258,-0.239939,0.593139,...,0.315912,0.796786,-0.060053,1.056942,0.509796,-0.181181,0.011037,-0.080467,-0.081839,0
284805,284806,1.641971,-0.122755,0.321250,0.463319,0.487192,-0.273836,0.468154,-0.554671,0.568630,...,0.361111,1.102449,-0.261503,0.203427,-1.091853,1.133633,0.269604,0.316686,-0.313248,0


In [16]:
# separar predictores y variable objetivo

X = fraude_preprocess.drop('Class', axis=1)
y = fraude_preprocess['Class']
y

0         0
1         0
2         0
3         0
4         0
         ..
284802    0
284803    0
284804    0
284805    0
284806    0
Name: Class, Length: 284807, dtype: int64

In [28]:
# tecnicas de oversampling: caso extremo clase minoritaria menor a 1%

smote = SMOTE(sampling_strategy= 0.4, n_jobs=-1,random_state = 100)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X, y)
y_resampled_smote

adasyn = ADASYN(sampling_strategy = 'auto', n_jobs=-1, random_state=100)
X_resampled_adasyn, y_resampled_adasyn = adasyn.fit_resample(X, y)
y_resampled_adasyn

borderline = BorderlineSMOTE(sampling_strategy = 'auto', n_jobs=-1, random_state=100)
X_resampled_borderline, y_resampled_borderline = borderline.fit_resample(X, y)
y_resampled_borderline

svm_SMOTE = SVMSMOTE(sampling_strategy = 'auto', n_jobs=-1, random_state=100)
X_resampled_svm_SMOTE, y_resampled_svm_SMOTE = svm_SMOTE.fit_resample(X, y)
y_resampled_svm_SMOTE

#kmeans_smote = KMeansSMOTE(sampling_strategy = 'auto', n_jobs=-1, random_state=100)
#X_resampled_kmeans, y_resampled_kmeans = kmeans_smote.fit_resample(X, y)
#y_resampled_kmeans


0         0
1         0
2         0
3         0
4         0
         ..
568625    1
568626    1
568627    1
568628    1
568629    1
Name: Class, Length: 568630, dtype: int64

In [ ]:
# split data in train-set 